In [4]:
import pandas as pd
import cloudscraper
import os
import requests
from bs4 import BeautifulSoup as bs
from datetime import timedelta, date
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import sys
import spotipy
import spotipy.util as util
import time

scope = 'user-library-read'
username = '38aiox0njoaisv8ldippfpbyc'

token = util.prompt_for_user_token(username,scope,client_id='40a9fe14bfc7475ab1aa160efe7674f5',client_secret='c6d0449bf1244ad09157d6f33bc32083',redirect_uri='https://www.google.com/')

if token:
    sp = spotipy.Spotify(auth=token)
    results = sp.current_user_saved_tracks()
    for item in results['items']:
        track = item['track']
        print(track['name'] + ' - ' + track['artists'][0]['name'])
else:
    print("Can't get token for", username)
    
# It reads the webpage.
def get_webpage(link):
    headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}
    page = requests.get(link, headers = headers, timeout = 100)
    soup = bs(page.content, 'html.parser')
    return(soup)

# It generates a list of dates between Jan 1, 2017 and today
# in YYYY-MM-DD format
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

# It creates the list of page links we will get the data from.
def create_links(country):
    start_date = date(2020, 3, 1)
    end_date = date(2020, 8, 1)
    links = []
    dates = daterange(start_date, end_date)
    for single_date in daterange(start_date, end_date):
        links.append('https://spotifycharts.com/regional/' + country + '/daily/' + single_date.strftime("%Y-%m-%d"))
    return(links, dates)

# It collects the data for each country, and write them in a list.
# The entries are (in order): Song, Artist, Date, Play Count, Rank
def get_data(country):
    [links, dates] = create_links(country)
    rows = []
    for (link, date) in zip(links, dates):
        soup = get_webpage(link)
        songlinks = soup.find_all("td", class_="chart-table-image")
        entries = soup.find_all("td", class_ = "chart-table-track")
        streams = soup.find_all("td", class_="chart-table-streams")
        for i, (songlink,entry, stream) in enumerate(zip(songlinks,entries,streams)):
            if i > 4:
                break
            song_url = songlink.find('a',href=True)['href']
            song = entry.find('strong').get_text()
            artist = entry.find('span').get_text()[3:]
            play_count = stream.get_text()
            country_name = country
            result = sp.search(song)
            track = result['tracks']['items'][0]
            artist_data = sp.artist(track["artists"][0]["external_urls"]["spotify"])
            genre = artist_data["genres"]
            print(song_url)
#             try:
            audio_features = sp.audio_features(song_url)
#             catch(e):
#                 print("SONG URL: ", song_url)
#                 continue
                
            danceability_key = "danceability"
            energy_key = "energy"
            loudness_key = "loudness"
            speechiness_key = "speechiness"
            acousticness_key = "acousticness"
            instrumentalness_key = "instrumentalness"
            liveness_key = "liveness"
            valence_key = "valence"
            tempo_key = "tempo"
            
            dance = [dance_dict[danceability_key] for dance_dict in audio_features]
            energy = [energy_dict[energy_key] for energy_dict in audio_features]
            loudness = [loudness_dict[loudness_key] for loudness_dict in audio_features]
            speech = [speechiness_dict[speechiness_key] for speechiness_dict in audio_features]
            acoustic = [acousticness_dict[acousticness_key] for acousticness_dict in audio_features]
            instrument = [instrumentalness_dict[instrumentalness_key] for instrumentalness_dict in audio_features]
            live = [liveness_dict[liveness_key] for liveness_dict in audio_features]
            valence = [valence_dict[valence_key] for valence_dict in audio_features]
            tempo = [tempo_dict[tempo_key] for tempo_dict in audio_features]
            
            dance = dance[0]
            energy = energy[0]
            loudness = loudness[0]
            speech = speech[0]
            acoustic = acoustic[0]
            instrument = instrument[0]
            live = live[0]
            valence = valence[0]
            tempo = tempo[0]
            lst = [song, artist, date, play_count, i+1, country_name, song_url, dance, energy, loudness, speech, acoustic, instrument, live, valence, tempo, genre]
            
            rows.append(lst)
            
            #print(i)
    return(rows)

# It exports the data for each country in a csv format.
# The column names are Song, Artist, Date, Streams, Rank.
def save_data(country):
    if not os.path.exists('data'):
        os.makedirs('data')
    file_name = 'data/' + country[1].replace(" ", "_").lower() + '.csv'
    data = get_data(country[0])
    if(len(data)!= 0):
        data = pd.DataFrame(data, columns=['Song','Artist','Date', 'Streams','Rank', 'Country', 'Links', 'Danceability', 'Energy', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo', 'Genre'])
        data.to_csv(file_name, sep=',', float_format='%s', index = False)

# It generates a list of countries for which the data is provided.
def get_countries():
    headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}
    page = requests.get('https://www.spotifycharts.com/regional', headers = headers)
    soup = bs(page.content, 'html.parser')
    countries = []
    ctys = soup.find('ul').findAll('li')
    for cty in ctys:
        if cty.get_text() == 'India':
            countries.append([cty["data-value"],cty.get_text()])
            
        if cty.get_text() == 'United States':
            countries.append([cty["data-value"],cty.get_text()])
            
        if cty.get_text() == 'United Kingdom':
            countries.append([cty["data-value"],cty.get_text()])
    return(countries)

# It runs the function save_data for each country.
# In other words, it creates the .csv data files for each country.
def scrape_data():
    countries = get_countries()
    print(countries)
    for country in countries:
        save_data(country)

scrape_data()

[['us', 'United States'], ['gb', 'United Kingdom'], ['in', 'India']]
https://open.spotify.com/track/0nbXyq5TXYPCO7pr3N8S4I
https://open.spotify.com/track/3Dv1eDb0MEgF93GpLXlucZ
https://open.spotify.com/track/0sf12qNH5qcw8qpgymFOqD
https://open.spotify.com/track/5yY9lUy8nbvjM1Uyo1Uqoc
https://open.spotify.com/track/22LAwLoDA5b4AaGSkg6bKW
https://open.spotify.com/track/0nbXyq5TXYPCO7pr3N8S4I
https://open.spotify.com/track/5raTDhlbxFDnpn89bxArdH
https://open.spotify.com/track/3Dv1eDb0MEgF93GpLXlucZ
https://open.spotify.com/track/0sf12qNH5qcw8qpgymFOqD
https://open.spotify.com/track/22LAwLoDA5b4AaGSkg6bKW
https://open.spotify.com/track/0nbXyq5TXYPCO7pr3N8S4I
https://open.spotify.com/track/5raTDhlbxFDnpn89bxArdH
https://open.spotify.com/track/22LAwLoDA5b4AaGSkg6bKW
https://open.spotify.com/track/0sf12qNH5qcw8qpgymFOqD
https://open.spotify.com/track/3Dv1eDb0MEgF93GpLXlucZ
https://open.spotify.com/track/0nbXyq5TXYPCO7pr3N8S4I
https://open.spotify.com/track/5raTDhlbxFDnpn89bxArdH
https://open.

https://open.spotify.com/track/22LAwLoDA5b4AaGSkg6bKW
https://open.spotify.com/track/3PfIrDoz19wz7qK7tYeu62
https://open.spotify.com/track/0lSZh5W0wDeurkGzLYY6hf
https://open.spotify.com/track/0VjIjW4GlUZAMYd2vXMi3b
https://open.spotify.com/track/0nbXyq5TXYPCO7pr3N8S4I
https://open.spotify.com/track/22LAwLoDA5b4AaGSkg6bKW
https://open.spotify.com/track/3PfIrDoz19wz7qK7tYeu62
https://open.spotify.com/track/0lSZh5W0wDeurkGzLYY6hf
https://open.spotify.com/track/0nbXyq5TXYPCO7pr3N8S4I
https://open.spotify.com/track/0VjIjW4GlUZAMYd2vXMi3b
https://open.spotify.com/track/22LAwLoDA5b4AaGSkg6bKW
https://open.spotify.com/track/1lGHa2pwYzxQHFBUynhLtO
https://open.spotify.com/track/3PfIrDoz19wz7qK7tYeu62
https://open.spotify.com/track/127QTOFJsJQp5LbJbu3A1y
https://open.spotify.com/track/0nbXyq5TXYPCO7pr3N8S4I
https://open.spotify.com/track/0VjIjW4GlUZAMYd2vXMi3b
https://open.spotify.com/track/22LAwLoDA5b4AaGSkg6bKW
https://open.spotify.com/track/1lGHa2pwYzxQHFBUynhLtO
https://open.spotify.com/tra

https://open.spotify.com/track/22LAwLoDA5b4AaGSkg6bKW
https://open.spotify.com/track/6Kj17Afjo1OKJYpf5VzCeo
https://open.spotify.com/track/466cKvZn1j45IpxDdYZqdA
https://open.spotify.com/track/39Yp9wwQiSRIDOvrVg7mbk
https://open.spotify.com/track/4wVOKKEHUJxHCFFNUWDn0B
https://open.spotify.com/track/5fHrHoJKfCgBd35zwIBmUK
https://open.spotify.com/track/39Yp9wwQiSRIDOvrVg7mbk
https://open.spotify.com/track/466cKvZn1j45IpxDdYZqdA
https://open.spotify.com/track/6Kj17Afjo1OKJYpf5VzCeo
https://open.spotify.com/track/5v4GgrXPMghOnBBLmveLac
https://open.spotify.com/track/7ytR5pFWmSjzHJIeQkgog4
https://open.spotify.com/track/39Yp9wwQiSRIDOvrVg7mbk
https://open.spotify.com/track/466cKvZn1j45IpxDdYZqdA
https://open.spotify.com/track/7ytR5pFWmSjzHJIeQkgog4
https://open.spotify.com/track/6Kj17Afjo1OKJYpf5VzCeo
https://open.spotify.com/track/5v4GgrXPMghOnBBLmveLac
https://open.spotify.com/track/39Yp9wwQiSRIDOvrVg7mbk
https://open.spotify.com/track/466cKvZn1j45IpxDdYZqdA
https://open.spotify.com/tra

https://open.spotify.com/track/39Yp9wwQiSRIDOvrVg7mbk
https://open.spotify.com/track/0VjIjW4GlUZAMYd2vXMi3b
https://open.spotify.com/track/466cKvZn1j45IpxDdYZqdA
https://open.spotify.com/track/7ytR5pFWmSjzHJIeQkgog4
https://open.spotify.com/track/7ju97lgwC2rKQ6wwsf9no9
https://open.spotify.com/track/39Yp9wwQiSRIDOvrVg7mbk
https://open.spotify.com/track/22LAwLoDA5b4AaGSkg6bKW
https://open.spotify.com/track/5RqR4ZCCKJDcBLIn4sih9l
https://open.spotify.com/track/7ytR5pFWmSjzHJIeQkgog4
https://open.spotify.com/track/0b9oOr2ZgvyQu88wzixux9
https://open.spotify.com/track/39Yp9wwQiSRIDOvrVg7mbk
https://open.spotify.com/track/5RqR4ZCCKJDcBLIn4sih9l
https://open.spotify.com/track/22LAwLoDA5b4AaGSkg6bKW
https://open.spotify.com/track/7ytR5pFWmSjzHJIeQkgog4
https://open.spotify.com/track/39Yp9wwQiSRIDOvrVg7mbk
https://open.spotify.com/track/5RqR4ZCCKJDcBLIn4sih9l
https://open.spotify.com/track/22LAwLoDA5b4AaGSkg6bKW
https://open.spotify.com/track/0VjIjW4GlUZAMYd2vXMi3b
https://open.spotify.com/tra

https://open.spotify.com/track/7mEUjhl6aTztdHCrxoTgBG
https://open.spotify.com/track/7ytR5pFWmSjzHJIeQkgog4
https://open.spotify.com/track/2MbdDtCv5LUVjYy9RuGTgC
https://open.spotify.com/track/5RqR4ZCCKJDcBLIn4sih9l
https://open.spotify.com/track/0VjIjW4GlUZAMYd2vXMi3b
https://open.spotify.com/track/22LAwLoDA5b4AaGSkg6bKW
https://open.spotify.com/track/7ytR5pFWmSjzHJIeQkgog4
https://open.spotify.com/track/2MbdDtCv5LUVjYy9RuGTgC
https://open.spotify.com/track/5RqR4ZCCKJDcBLIn4sih9l
https://open.spotify.com/track/6UelLqGlWMcVH1E5c4H7lY
https://open.spotify.com/track/22LAwLoDA5b4AaGSkg6bKW
https://open.spotify.com/track/7ytR5pFWmSjzHJIeQkgog4
https://open.spotify.com/track/0PvFJmanyNQMseIFrU708S
https://open.spotify.com/track/1H7KnK26kc1YyellpbINEn
https://open.spotify.com/track/6O7fPYTsb2mY0PiFt2ymDB
https://open.spotify.com/track/2MbdDtCv5LUVjYy9RuGTgC
https://open.spotify.com/track/7ytR5pFWmSjzHJIeQkgog4
https://open.spotify.com/track/0PvFJmanyNQMseIFrU708S
https://open.spotify.com/tra

...

https://open.spotify.com/track/4umIPjkehX1r7uhmGvXiSV
https://open.spotify.com/track/2rRJrJEo19S2J82BDsQ3F7
https://open.spotify.com/track/0VjIjW4GlUZAMYd2vXMi3b
https://open.spotify.com/track/0gzu5mm36VJH2Zqu8sQPTf
https://open.spotify.com/track/1smFN2CLqGROu0J0UyvDfL
https://open.spotify.com/track/4umIPjkehX1r7uhmGvXiSV
https://open.spotify.com/track/2rRJrJEo19S2J82BDsQ3F7
https://open.spotify.com/track/0VjIjW4GlUZAMYd2vXMi3b
https://open.spotify.com/track/0gzu5mm36VJH2Zqu8sQPTf
https://open.spotify.com/track/1smFN2CLqGROu0J0UyvDfL
https://open.spotify.com/track/4umIPjkehX1r7uhmGvXiSV
https://open.spotify.com/track/2rRJrJEo19S2J82BDsQ3F7
https://open.spotify.com/track/0VjIjW4GlUZAMYd2vXMi3b
https://open.spotify.com/track/0gzu5mm36VJH2Zqu8sQPTf
https://open.spotify.com/track/1smFN2CLqGROu0J0UyvDfL
https://open.spotify.com/track/2rRJrJEo19S2J82BDsQ3F7
https://open.spotify.com/track/4umIPjkehX1r7uhmGvXiSV
https://open.spotify.com/track/0VjIjW4GlUZAMYd2vXMi3b
https://open.spotify.com/tra

https://open.spotify.com/track/127QTOFJsJQp5LbJbu3A1y
https://open.spotify.com/track/0gzu5mm36VJH2Zqu8sQPTf
https://open.spotify.com/track/1smFN2CLqGROu0J0UyvDfL
https://open.spotify.com/track/7eJMfftS33KTjuF7lTsMCx
https://open.spotify.com/track/2rRJrJEo19S2J82BDsQ3F7
https://open.spotify.com/track/0WdbnNKO0Jt4BZACSDQh44
https://open.spotify.com/track/0gzu5mm36VJH2Zqu8sQPTf
https://open.spotify.com/track/1smFN2CLqGROu0J0UyvDfL
https://open.spotify.com/track/7eJMfftS33KTjuF7lTsMCx
https://open.spotify.com/track/466cKvZn1j45IpxDdYZqdA
https://open.spotify.com/track/0WdbnNKO0Jt4BZACSDQh44
https://open.spotify.com/track/0gzu5mm36VJH2Zqu8sQPTf
https://open.spotify.com/track/1smFN2CLqGROu0J0UyvDfL
https://open.spotify.com/track/7eJMfftS33KTjuF7lTsMCx
https://open.spotify.com/track/466cKvZn1j45IpxDdYZqdA
https://open.spotify.com/track/2rRJrJEo19S2J82BDsQ3F7
https://open.spotify.com/track/0gzu5mm36VJH2Zqu8sQPTf
https://open.spotify.com/track/1smFN2CLqGROu0J0UyvDfL
https://open.spotify.com/tra

https://open.spotify.com/track/7eJMfftS33KTjuF7lTsMCx
https://open.spotify.com/track/0gzu5mm36VJH2Zqu8sQPTf
https://open.spotify.com/track/2Fv2injs4qAm8mJBGaxVKU
https://open.spotify.com/track/1smFN2CLqGROu0J0UyvDfL
https://open.spotify.com/track/4DpNNXFMMxQEKl7r0ykkWA
https://open.spotify.com/track/7eJMfftS33KTjuF7lTsMCx
https://open.spotify.com/track/0gzu5mm36VJH2Zqu8sQPTf
https://open.spotify.com/track/2Fv2injs4qAm8mJBGaxVKU
https://open.spotify.com/track/1smFN2CLqGROu0J0UyvDfL
https://open.spotify.com/track/4DpNNXFMMxQEKl7r0ykkWA
https://open.spotify.com/track/7eJMfftS33KTjuF7lTsMCx
https://open.spotify.com/track/0gzu5mm36VJH2Zqu8sQPTf
https://open.spotify.com/track/2Fv2injs4qAm8mJBGaxVKU
https://open.spotify.com/track/1smFN2CLqGROu0J0UyvDfL
https://open.spotify.com/track/4DpNNXFMMxQEKl7r0ykkWA
https://open.spotify.com/track/7eJMfftS33KTjuF7lTsMCx
https://open.spotify.com/track/0gzu5mm36VJH2Zqu8sQPTf
https://open.spotify.com/track/2Fv2injs4qAm8mJBGaxVKU
https://open.spotify.com/tra

https://open.spotify.com/track/1xQ6trAsedVPCdbtDAmk0c
https://open.spotify.com/track/3vAn0qZzdyuHamcrpkfiX3
https://open.spotify.com/track/1smFN2CLqGROu0J0UyvDfL
https://open.spotify.com/track/5O932cZmzOZGOGZz9RHx20
https://open.spotify.com/track/4DpNNXFMMxQEKl7r0ykkWA
https://open.spotify.com/track/1xQ6trAsedVPCdbtDAmk0c
https://open.spotify.com/track/1smFN2CLqGROu0J0UyvDfL
https://open.spotify.com/track/5O932cZmzOZGOGZz9RHx20
https://open.spotify.com/track/4DpNNXFMMxQEKl7r0ykkWA
https://open.spotify.com/track/3vAn0qZzdyuHamcrpkfiX3
https://open.spotify.com/track/1xQ6trAsedVPCdbtDAmk0c
https://open.spotify.com/track/1smFN2CLqGROu0J0UyvDfL
https://open.spotify.com/track/5O932cZmzOZGOGZz9RHx20
https://open.spotify.com/track/4DpNNXFMMxQEKl7r0ykkWA
https://open.spotify.com/track/1xQ6trAsedVPCdbtDAmk0c
https://open.spotify.com/track/3vAn0qZzdyuHamcrpkfiX3
https://open.spotify.com/track/1smFN2CLqGROu0J0UyvDfL
https://open.spotify.com/track/5O932cZmzOZGOGZz9RHx20
https://open.spotify.com/tra

https://open.spotify.com/track/7krGSbOnw13S7tIpgVEWAo
https://open.spotify.com/track/1smFN2CLqGROu0J0UyvDfL
https://open.spotify.com/track/4UJkUnfEEL8B2JZ1OQfYOO
https://open.spotify.com/track/46QbY78ha62aiu6gBgC7lS
https://open.spotify.com/track/2ZvWoJ82LKCgD0nU9H5ue5
https://open.spotify.com/track/1smFN2CLqGROu0J0UyvDfL
https://open.spotify.com/track/1xQ6trAsedVPCdbtDAmk0c
https://open.spotify.com/track/4UJkUnfEEL8B2JZ1OQfYOO


In [ ]:
print("done")